# Zakharov function
---
Description:

- Optimization (min)
- Single-objective
- Constraints (no)
---

Minimize the equation given by:

- $f\left(\mathbf{x}\right) = \sum_{i=1}^{d}x_i^2 + \left(\sum_{i=1}^{d}0.5ix_i\right)^2 + \left(\sum_{i=1}^{d}0.5ix_i\right)^4$

where: $-10 \le x_i \le 10$.

Global Minimum:

- $f\left(\mathbf{\hat{x}}\right) = 0$, at $\mathbf{\hat{x}} = [0, 0, ..., 0]$

## First we import python libraries and set up the directory of our code

In [1]:
import os, sys
import numpy as np
from math import isclose

PROJECT_DIR = os.path.abspath('..')
sys.path.append(PROJECT_DIR)

## Here we import all our custom PSO code

In [2]:
from star_pso.auxiliary.swarm import Swarm
from star_pso.auxiliary.particle import Particle
from star_pso.engines.standard_pso import StandardPSO

## Define the objective function

In [3]:
# Objective function.
def fun_Zakharov(x: np.typing.ArrayLike):

    # Create an array of indices.
    indices = np.arange(len(x))

    # Compute the sum of 'i * x_i'.
    sum_xi = 0.5*np.sum(indices * x)
        
    # Compute the final value.
    f_val = np.sum(x**2) + sum_xi**2 + sum_xi**4

    # Condition for termination.
    solution_found = isclose(f_val, 0.0, rel_tol=1.0e-6)

    # Return the solution tuple.
    return -f_val, solution_found
# _end_def_

## Here we set the PSO parameters

- Set the number of dimensions 'D'
- Set the number of particles 'N'
- Setup the initial population
- Create the PSO object

In [4]:
# Random number generator.
rng = np.random.default_rng()

# Define the number of optimizing variables.
D = 8

# Define the number of particles.
N = min(5*D, 100)

# Sample the initial points randomly.
X_t0 = rng.uniform(-10.0, +10.0, size=(N, D))

# Initial population.
swarm_t0 = Swarm([Particle(x) for x in X_t0])

# Create the StandardPSO object that will carry on the optimization.
test_PSO = StandardPSO(initial_swarm= swarm_t0, obj_func= fun_Zakharov,
                       x_min= -10.0, x_max= +10.0)

## Optimization process

In [5]:
test_PSO.run(max_it = 200,
             f_tol = 1.0e-5,
             options = {"w": 0.7, "c1": 1.56, "c2": 1.05},
             reset_swarm = True,
             verbose = True)

Initial f_optimal = -104.0869
Iteration:     1 -> f_optimal = -52.0916
Iteration:    21 -> f_optimal = -4.2857
Iteration:    41 -> f_optimal = -0.7702
Iteration:    61 -> f_optimal = -0.0183
Iteration:    81 -> f_optimal = -0.0004
Iteration:   101 -> f_optimal = -0.0000
Iteration:   121 -> f_optimal = -0.0000
Iteration:   141 -> f_optimal = -0.0000
Iteration:   161 -> f_optimal = -0.0000
Iteration:   181 -> f_optimal = -0.0000
Final f_optimal = -0.0000
run: elapsed time = 0.239 seconds.


In [6]:
# Extract the optimal solution from the PSO.
optimal_solution = test_PSO.swarm.best_particle()

# Display the (final) optimal value.
print(f"Optimum Found: {optimal_solution.value:.6f}\n")

# Display each gene value separately.
for i, xi in enumerate(optimal_solution.position, start=1):
    print(f"x{i} = {xi:>10.6f}")
# _end_for_

Optimum Found: -0.000000

x1 =  -0.000001
x2 =  -0.000000
x3 =   0.000000
x4 =   0.000000
x5 =  -0.000000
x6 =  -0.000001
x7 =   0.000001
x8 =   0.000000


### End of file